# minbpe: Minimal Byte Pair Encoding Tokenizer

This notebook contains a faithful implementation of Andrej Karpathy's minbpe tokenizer, emphasizing:
- Clean, minimal code
- Byte-level tokenization
- Educational clarity

The implementation follows the core design philosophy of Karpathy's approach, preserving the simplicity and readability of the original.

In [1]:
import os
import json
import time
import regex as re
from collections import Counter
from typing import List, Dict, Tuple, Optional, Set, Any, Union

## Base Tokenizer Implementation

We start with a basic tokenizer class that implements the core BPE algorithm without any regex-based preprocessing.

In [2]:
class Tokenizer:
    """A minimal Byte Pair Encoding tokenizer implementation."""
    
    def __init__(self):
        # Initialize with the base 256 tokens (raw bytes 0-255)
        self.merges = {}  # (token1, token2) -> new_token_id 
        self.vocab = {}   # token_id -> token (bytes)
        self.vocab_size = 0
        self.special_tokens = {}
        
        # Pre-populate the vocabulary with the basic 256 byte tokens
        for i in range(256):
            token = bytes([i])
            self.vocab[i] = token
            
        self.vocab_size = 256
    
    def train(self, text: str, vocab_size: int, verbose: bool = False) -> None:
        """Train the tokenizer on text, extending the vocabulary to the desired size."""
        # Convert text to bytes
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes)
        
        # Keep track of progress
        if verbose:
            print(f"Training BPE tokenizer to vocab size {vocab_size}")
            print(f"Text size: {len(text)} chars, {len(ids)} bytes")
        
        # Iteratively merge the most frequent pair until we reach the desired vocab size
        num_merges = vocab_size - 256
        for i in range(num_merges):
            # Count frequencies of adjacent pairs
            stats = self.get_stats(ids)
            if not stats:
                break
                
            # Find the most frequent pair
            pair = max(stats, key=stats.get)
            
            # Create a new token for this pair
            token1, token2 = pair
            new_token = self.vocab[token1] + self.vocab[token2]
            new_id = self.vocab_size
            
            # Add merge to our vocabulary
            self.merges[pair] = new_id
            self.vocab[new_id] = new_token
            self.vocab_size += 1
            
            # Apply the merge to the current token list
            ids = self.merge(ids, pair, new_id)
            
            # Print progress
            if verbose and i % 100 == 0:
                print(f"Merge #{i}: pair {pair} -> {new_id}, corpus now {len(ids)} tokens")
    
    def get_stats(self, ids: List[int]) -> Dict[Tuple[int, int], int]:
        """Count the frequencies of adjacent token pairs."""
        stats = Counter()
        for i in range(len(ids) - 1):
            pair = (ids[i], ids[i+1])
            stats[pair] += 1
        return stats
    
    def merge(self, ids: List[int], pair: Tuple[int, int], new_id: int) -> List[int]:
        """Replace all occurrences of a token pair with a new token ID."""
        # Create a new list for the merged result
        new_ids = []
        i = 0
        while i < len(ids):
            # If we're at the last token, just add it
            if i == len(ids) - 1:
                new_ids.append(ids[i])
                break
            
            # If current pair matches, merge and add the new token
            if ids[i] == pair[0] and ids[i+1] == pair[1]:
                new_ids.append(new_id)
                i += 2  # Skip both tokens
            else:
                new_ids.append(ids[i])
                i += 1  # Move to next token
        
        return new_ids
    
    def encode(self, text: str) -> List[int]:
        """Encode text to token IDs."""
        # Convert text to bytes
        text_bytes = text.encode("utf-8")
        ids = list(text_bytes)
        
        # Apply merges iteratively, in the order they were learned
        while len(ids) >= 2:
            # Find valid merge pairs in the current sequence
            pairs = [(ids[i], ids[i+1]) for i in range(len(ids)-1)]
            valid_pairs = [(pair, self.merges[pair]) for pair in pairs if pair in self.merges]
            
            # If no valid pairs, we're done
            if not valid_pairs:
                break
                
            # Find the pair with the lowest merge ID (first learned)
            pair, new_id = min(valid_pairs, key=lambda x: x[1])
            
            # Apply the merge
            ids = self.merge(ids, pair, new_id)
        
        return ids
    
    def decode(self, ids: List[int]) -> str:
        """Decode token IDs back to text."""
        # Convert token IDs to bytes
        bytes_list = []
        for token_id in ids:
            bytes_list.extend(self.vocab[token_id])
        
        # Convert bytes to UTF-8 text
        text = bytes(bytes_list).decode("utf-8", errors="replace")
        return text
    
    def save(self, file_path: str) -> None:
        """Save the tokenizer to a file."""
        # Prepare model data - convert bytes to lists for JSON serialization
        model_data = {
            "vocab_size": self.vocab_size,
            "merges": {f"{t1},{t2}": idx for (t1, t2), idx in self.merges.items()},
            "vocab": {str(i): list(t) for i, t in self.vocab.items() if i >= 256},
            "special_tokens": self.special_tokens
        }
        
        # Save to file
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(model_data, f, ensure_ascii=False, indent=2)
    
    def load(self, file_path: str) -> None:
        """Load a tokenizer from a file."""
        # Read the model data
        with open(file_path, 'r', encoding='utf-8') as f:
            model_data = json.load(f)
        
        # Reset the tokenizer
        self.__init__()
        
        # Load the vocabulary
        self.vocab_size = model_data["vocab_size"]
        
        # Add vocabulary items (skipping the base 256 bytes already initialized)
        for token_id_str, token_bytes in model_data["vocab"].items():
            token_id = int(token_id_str)
            self.vocab[token_id] = bytes(token_bytes)
        
        # Load merges
        for pair_str, idx in model_data["merges"].items():
            t1, t2 = map(int, pair_str.split(","))
            self.merges[(t1, t2)] = idx
        
        # Load special tokens
        self.special_tokens = model_data.get("special_tokens", {})
    
    def token_to_str(self, token_id: int) -> str:
        """Get a string representation of a token for visualization."""
        token_bytes = self.vocab[token_id]
        # Try to convert to UTF-8 string if possible
        try:
            s = token_bytes.decode('utf-8')
            # Replace newlines, tabs, etc. for display
            s = s.replace('\n', '\\n').replace('\t', '\\t')
            if len(s.strip()) == 0:
                # If it's all whitespace, show hex
                return f"[hex: {token_bytes.hex()}]"
            return s
        except UnicodeDecodeError:
            # If not a valid UTF-8 string, show hex
            return f"[hex: {token_bytes.hex()}]"
    
    def print_vocab(self, n=50) -> None:
        """Print the first n tokens in the vocabulary for inspection."""
        ids = sorted(self.vocab.keys())
        skipped = max(0, len(ids) - n)
        print(f"Vocabulary size: {len(ids)} tokens")
        print(f"Showing first {min(n, len(ids))} tokens:")
        for i, token_id in enumerate(ids[:n]):
            s = self.token_to_str(token_id)
            print(f"Token {token_id}: {s}")
        if skipped > 0:
            print(f"... and {skipped} more tokens")

## Testing the Tokenizer

Let's test our implementation with a simple example text.

In [3]:
# Create a simple training corpus
training_text = """
Byte Pair Encoding (BPE) is a data compression technique that iteratively replaces the most frequent pair of consecutive bytes in a sequence with a single, unused byte. In NLP, it is used as a subword tokenization algorithm.

The BPE algorithm works as follows:
1. Initialize the vocabulary with individual characters/bytes
2. Count all pairs of adjacent symbols in the training corpus
3. Merge the most frequent pair and add it to the vocabulary
4. Repeat steps 2-3 until reaching the desired vocabulary size

BPE can handle out-of-vocabulary words by splitting them into known subword units, making it effective for various languages and even emoji 👍🌍.
"""

# Initialize our tokenizer
tokenizer = Tokenizer()

# Train to a vocabulary size of 500
tokenizer.train(training_text, vocab_size=500, verbose=True)

# Show some of the learned tokens
tokenizer.print_vocab(30)

Training BPE tokenizer to vocab size 500
Text size: 656 chars, 662 bytes
Merge #0: pair (101, 32) -> 256, corpus now 647 tokens
Merge #100: pair (355, 110) -> 356, corpus now 311 tokens
Merge #200: pair (455, 277) -> 456, corpus now 211 tokens
Vocabulary size: 500 tokens
Showing first 30 tokens:
Token 0:  
Token 1: 
Token 2: 
Token 3: 
Token 4: 
Token 5: 
Token 6: 
Token 7: 
Token 8:
Token 9: \t
Token 10: \n
Token 11: [hex: 0b]
Token 12: [hex: 0c]
Token 13: [hex: 0d]
Token 14: 
Token 15: 
Token 16: 
Token 17: 
Token 18: 
Token 19: 
Token 20: 
Token 21: 
Token 22: 
Token 23: 
Token 24: 
Token 25: 
Token 26: 
Token 27: 
Token 28: [hex: 1c]
Token 29: [hex: 1d]
... and 470 more tokens


## Encoding and Decoding

Now let's test encoding and decoding to verify the tokenizer works as expected.

In [4]:
# Test with a new sentence
test_text = "BPE tokenization works great for natural language processing!"

# Encode the text
encoded = tokenizer.encode(test_text)
print(f"Encoded into {len(encoded)} tokens: {encoded}")

# Display each token
print("\nToken breakdown:")
for i, token_id in enumerate(encoded):
    print(f"Token {i+1}: ID {token_id} = '{tokenizer.token_to_str(token_id)}'")

# Decode the tokens back to text
decoded = tokenizer.decode(encoded)
print(f"\nDecoded text: '{decoded}'")
print(f"Round trip success: {test_text == decoded}")

Encoded into 42 tokens: [292, 32, 305, 107, 269, 306, 97, 266, 111, 293, 282, 107, 257, 103, 265, 311, 102, 262, 32, 110, 97, 116, 117, 295, 108, 32, 267, 110, 103, 117, 97, 103, 256, 112, 114, 111, 99, 101, 115, 115, 264, 33]

Token breakdown:
Token 1: ID 292 = 'BPE'
Token 2: ID 32 = '[hex: 20]'
Token 3: ID 305 = 'to'
Token 4: ID 107 = 'k'
Token 5: ID 269 = 'en'
Token 6: ID 306 = 'iz'
Token 7: ID 97 = 'a'
Token 8: ID 266 = 'ti'
Token 9: ID 111 = 'o'
Token 10: ID 293 = 'n '
Token 11: ID 282 = 'wor'
Token 12: ID 107 = 'k'
Token 13: ID 257 = 's '
Token 14: ID 103 = 'g'
Token 15: ID 265 = 're'
Token 16: ID 311 = 'at '
Token 17: ID 102 = 'f'
Token 18: ID 262 = 'or'
Token 19: ID 32 = '[hex: 20]'
Token 20: ID 110 = 'n'
Token 21: ID 97 = 'a'
Token 22: ID 116 = 't'
Token 23: ID 117 = 'u'
Token 24: ID 295 = 'ra'
Token 25: ID 108 = 'l'
Token 26: ID 32 = '[hex: 20]'
Token 27: ID 267 = 'la'
Token 28: ID 110 = 'n'
Token 29: ID 103 = 'g'
Token 30: ID 117 = 'u'
Token 31: ID 97 = 'a'
Token 32: ID 103 

## Measuring Tokenization Efficiency

Let's compute some metrics on the tokenization efficiency.

In [5]:
def measure_efficiency(tokenizer, texts):
    """Measure tokenization efficiency across multiple text samples."""
    results = []
    for name, text in texts.items():
        # Tokenize and measure
        start_time = time.time()
        tokens = tokenizer.encode(text)
        encode_time = time.time() - start_time
        
        start_time = time.time()
        decoded = tokenizer.decode(tokens)
        decode_time = time.time() - start_time
        
        # Calculate metrics
        char_count = len(text)
        token_count = len(tokens)
        compression_ratio = char_count / token_count
        chars_per_second = char_count / encode_time if encode_time > 0 else 0
        
        # Store results
        results.append({
            "name": name,
            "chars": char_count,
            "tokens": token_count,
            "ratio": compression_ratio,
            "encode_time": encode_time,
            "decode_time": decode_time,
            "chars_per_second": chars_per_second,
            "roundtrip_success": text == decoded
        })
    
    # Print results table
    print(f"{'Text':<15} | {'Chars':<8} | {'Tokens':<8} | {'Ratio':<7} | {'Encode (s)':<10} | {'Decode (s)':<10} | {'Success':<7}")
    print("-" * 75)
    for r in results:
        print(f"{r['name']:<15} | {r['chars']:<8} | {r['tokens']:<8} | {r['ratio']:<7.2f} | {r['encode_time']:<10.4f} | {r['decode_time']:<10.4f} | {r['roundtrip_success']}")
    
    return results

# Define test texts
test_texts = {
    "English": "The quick brown fox jumps over the lazy dog.",
    "Repeated": "hello hello hello hello hello hello hello",
    "Numbers": "1234567890 1234567890 1234567890",
    "Technical": "def factorial(n): return 1 if n <= 1 else n * factorial(n-1)",
    "Emoji": "🙂 🌍 🚀 👨‍👩‍👧‍👦 🎉",
    "Mixed": "Training at 3.5x speed: 😊 快速训练！速度提高"
}

# Measure tokenization efficiency
efficiency_results = measure_efficiency(tokenizer, test_texts)

Text            | Chars    | Tokens   | Ratio   | Encode (s) | Decode (s) | Success
---------------------------------------------------------------------------
English         | 44       | 35       | 1.26    | 0.0001     | 0.0000     | True
Repeated        | 41       | 41       | 1.00    | 0.0000     | 0.0000     | True
Numbers         | 32       | 32       | 1.00    | 0.0000     | 0.0000     | True
Technical       | 60       | 50       | 1.20    | 0.0001     | 0.0000     | True
Emoji           | 15       | 37       | 0.41    | 0.0000     | 0.0000     | True
Mixed           | 35       | 48       | 0.73    | 0.0001     | 0.0000     | True


## Visualizing Tokenization Process

Let's create a visualization of how text gets split into tokens.

In [6]:
def visualize_tokenization(tokenizer, text):
    """Visualize how text is tokenized by showing token boundaries."""
    # Encode the text
    ids = tokenizer.encode(text)
    
    # Get the bytes for each token
    token_bytes = [tokenizer.vocab[id] for id in ids]
    
    # Try to display each token as text
    visualized = []
    for token in token_bytes:
        try:
            token_text = token.decode('utf-8')
            # Replace whitespace for visibility
            token_text = token_text.replace(' ', '␣').replace('\n', '\\n').replace('\t', '\\t')
            visualized.append(token_text)
        except UnicodeDecodeError:
            # If not a valid UTF-8 sequence, show hex
            visualized.append(f"[{token.hex()}]")
    
    # Display with token boundaries
    print(f"Tokenized into {len(ids)} tokens:")
    result = ""
    for token in visualized:
        result += f"[{token}]"
    print(result)
    
    # Display each token with its ID
    print("\nDetailed token breakdown:")
    for i, (id, vis) in enumerate(zip(ids, visualized)):
        print(f"Token {i+1}: ID {id} = '{vis}'")
    
    return visualized

# Visualize tokenization for a sample text
sample_text = "Hello, world! This is a test of BPE tokenization."
tokens_visualized = visualize_tokenization(tokenizer, sample_text)

Tokenized into 34 tokens:
[H][e][l][l][o][,␣][wor][l][d][!][␣][T][h][i][s␣][i][s␣a][␣][te][s][t␣][of][␣][BPE][␣][to][k][en][iz][a][ti][o][n][.]

Detailed token breakdown:
Token 1: ID 72 = 'H'
Token 2: ID 101 = 'e'
Token 3: ID 108 = 'l'
Token 4: ID 108 = 'l'
Token 5: ID 111 = 'o'
Token 6: ID 303 = ',␣'
Token 7: ID 282 = 'wor'
Token 8: ID 108 = 'l'
Token 9: ID 100 = 'd'
Token 10: ID 33 = '!'
Token 11: ID 32 = '␣'
Token 12: ID 84 = 'T'
Token 13: ID 104 = 'h'
Token 14: ID 105 = 'i'
Token 15: ID 257 = 's␣'
Token 16: ID 105 = 'i'
Token 17: ID 277 = 's␣a'
Token 18: ID 32 = '␣'
Token 19: ID 263 = 'te'
Token 20: ID 115 = 's'
Token 21: ID 260 = 't␣'
Token 22: ID 300 = 'of'
Token 23: ID 32 = '␣'
Token 24: ID 292 = 'BPE'
Token 25: ID 32 = '␣'
Token 26: ID 305 = 'to'
Token 27: ID 107 = 'k'
Token 28: ID 269 = 'en'
Token 29: ID 306 = 'iz'
Token 30: ID 97 = 'a'
Token 31: ID 266 = 'ti'
Token 32: ID 111 = 'o'
Token 33: ID 110 = 'n'
Token 34: ID 46 = '.'


## Saving and Loading the Tokenizer

Let's test the serialization and deserialization of our tokenizer.

In [7]:
# Save the tokenizer
tokenizer.save("bpe_tokenizer.json")
print(f"Saved tokenizer with {tokenizer.vocab_size} tokens")

# Create a new tokenizer and load the saved model
new_tokenizer = Tokenizer()
new_tokenizer.load("bpe_tokenizer.json")
print(f"Loaded tokenizer with {new_tokenizer.vocab_size} tokens")

# Verify the loaded tokenizer works the same
check_text = "Testing if the loaded tokenizer works correctly."
original_tokens = tokenizer.encode(check_text)
loaded_tokens = new_tokenizer.encode(check_text)

print(f"Original tokenizer: {len(original_tokens)} tokens")
print(f"Loaded tokenizer: {len(loaded_tokens)} tokens")
print(f"Tokens match: {original_tokens == loaded_tokens}")

Saved tokenizer with 500 tokens
Loaded tokenizer with 500 tokens
Original tokenizer: 32 tokens
Loaded tokenizer: 32 tokens
Tokens match: True


## Extension: Implementing GPT-style Regex Pre-tokenization

For a more advanced implementation, we can add regex pre-tokenization similar to GPT tokenizers.

In [9]:
class RegexTokenizer(Tokenizer):
    """Enhanced tokenizer with regex-based pre-tokenization like GPT models."""
    
    def __init__(self):
        super().__init__()
        # GPT-style regex pattern for splitting text before BPE
        self.pattern = re.compile(r"'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+")
    
    def encode(self, text: str) -> List[int]:
        """Encode text with regex pre-tokenization before applying BPE."""
        # First split text using regex pattern
        segments = [m.group() for m in re.finditer(self.pattern, text)]
        
        # Apply BPE encoding to each segment
        all_tokens = []
        for segment in segments:
            # Use the parent class's encoding logic for each segment
            segment_tokens = super().encode(segment)
            all_tokens.extend(segment_tokens)
            
        return all_tokens
    
    def save(self, file_path: str) -> None:
        """Save with additional regex pattern information."""
        # Get base data
        super().save(file_path)
        
        # Add regex pattern to the saved data
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        data['pattern'] = self.pattern.pattern
        
        with open(file_path, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)
    
    def load(self, file_path: str) -> None:
        """Load with regex pattern information."""
        super().load(file_path)
        
        # Load regex pattern if available
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        
        if 'pattern' in data:
            self.pattern = re.compile(data['pattern'])

## Testing the Regex Tokenizer

In [10]:
# Train the regex tokenizer
regex_tokenizer = RegexTokenizer()
regex_tokenizer.train(training_text, vocab_size=500, verbose=True)

# Compare with the basic tokenizer
compare_text = "It's not just tokenization, it's BPE tokenization with regex pre-processing!"

basic_tokens = tokenizer.encode(compare_text)
regex_tokens = regex_tokenizer.encode(compare_text)

print(f"Basic tokenizer: {len(basic_tokens)} tokens")
print(f"Regex tokenizer: {len(regex_tokens)} tokens")

# Visualize the differences
print("\nBasic tokenization:")
visualize_tokenization(tokenizer, compare_text)

print("\nRegex tokenization:")
visualize_tokenization(regex_tokenizer, compare_text)

Training BPE tokenizer to vocab size 500
Text size: 656 chars, 662 bytes
Merge #0: pair (101, 32) -> 256, corpus now 647 tokens
Merge #100: pair (355, 110) -> 356, corpus now 311 tokens
Merge #200: pair (455, 277) -> 456, corpus now 211 tokens
Basic tokenizer: 51 tokens
Regex tokenizer: 57 tokens

Basic tokenization:
Tokenized into 51 tokens:
[I][t]['][s␣][n][o][t␣][j][u][s][t␣][to][k][en][iz][a][ti][o][n][,␣][it]['][s␣][BPE][␣][to][k][en][iz][a][ti][o][n␣][with][␣re][g][e][x][␣][p][re][-][p][r][o][c][e][s][s][ing][!]

Detailed token breakdown:
Token 1: ID 73 = 'I'
Token 2: ID 116 = 't'
Token 3: ID 39 = '''
Token 4: ID 257 = 's␣'
Token 5: ID 110 = 'n'
Token 6: ID 111 = 'o'
Token 7: ID 260 = 't␣'
Token 8: ID 106 = 'j'
Token 9: ID 117 = 'u'
Token 10: ID 115 = 's'
Token 11: ID 260 = 't␣'
Token 12: ID 305 = 'to'
Token 13: ID 107 = 'k'
Token 14: ID 269 = 'en'
Token 15: ID 306 = 'iz'
Token 16: ID 97 = 'a'
Token 17: ID 266 = 'ti'
Token 18: ID 111 = 'o'
Token 19: ID 110 = 'n'
Token 20: ID 303 

['I',
 't',
 "'",
 's',
 '␣',
 'n',
 'o',
 't',
 '␣',
 'j',
 'u',
 's',
 't',
 '␣',
 'to',
 'k',
 'en',
 'iz',
 'a',
 'ti',
 'o',
 'n',
 ',',
 '␣',
 'it',
 "'",
 's',
 '␣',
 'BPE',
 '␣',
 'to',
 'k',
 'en',
 'iz',
 'a',
 'ti',
 'o',
 'n',
 '␣',
 'with',
 '␣re',
 'g',
 'e',
 'x',
 '␣',
 'p',
 're',
 '-',
 'p',
 'r',
 'o',
 'c',
 'e',
 's',
 's',
 'ing',
 '!']

## Conclusion

This notebook has demonstrated a faithful implementation of Karpathy's minbpe approach, featuring:

1. A minimal, byte-level BPE tokenizer
2. Clean, educational code structure
3. Complete training, encoding, and decoding functionality
4. Visualization tools for understanding the tokenization process
5. An advanced regex-based tokenizer extension

The implementation preserves the essential characteristics of Karpathy's original: simplicity, readability, and educational value.